In [1]:
import os, glob, shutil, argparse, re
import pandas as pd
from pathlib import Path
import numpy as np
import training_info

logdir=training_info.logdir
dir_pattern = logdir+'/{}/{}/{}/{}/epoch{}/b{}/worker{}/thread{}/'
aug_pattern = logdir+'/{}/{}/'
simple_dir_pattern = logdir+'/{}'
logtypes=training_info.logtypes
datasets=training_info.datasets
models=training_info.models
term=training_info.term
output_file="pid.log"

In [2]:
def get_dirname():
    dir_names = {}
    for logtype in logtypes:
        dir_names[logtype]=glob.glob(simple_dir_pattern.format(logtype)+f"/*/*/*/*/*/*/*/")

    return dir_names

def replace_str(target):
    target=target.replace('\n', '')
    target=target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num=replace_str(arr[arr.index(target)+jumpto])
    except:
        print(arr, target, arr[arr.index(target)+jumpto])
        num='NA'
    return num

parsed_dir=training_info.parsed_dir+"/cpu"
os.makedirs(parsed_dir, exist_ok=True)

breakdown_col_name=[]
for i in range(24):
    breakdown_col_name.append(f"CPU{i} Util (%)")
    
breakdown_col_name.append(f"Socket0 Util (%)")
breakdown_col_name.append(f"Socket1 Util (%)")
breakdown_col_name.append(f"Total CPU Util (%)")
dir_names=get_dirname()
print(dir_names)

{'train_only': ['./log/DDP/train_only/1024batch10/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/train_only/1024batch/default/resnet18/epoch5/b256/worker24/thread0/'], 'loadNtrain': ['./log/DDP/loadNtrain/size5/default/resnet18/epoch5/b256/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker2/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b1024/worker24/thread0/', './log/DDP/loadNtrain/1024batch10/default/resnet18/epoch5/b256/worker24/thread0/', './log/DDP/loadNtrain/1024batch/default/resnet18/epoch5/b1024/worker2/thread0/'], 'prepNloadNtrain': ['./log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker4/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet18/epoch5/b1024/worker16/thread0/', './log/DDP/prepNloadNtrain/size5/default/resnet1

In [3]:
def parser(dir_names, datasets):
    for logtype in logtypes:
        for logdir in dir_names[logtype]:
            if not os.path.exists(logdir):
                    continue
            logdir_list=logdir.split('/')
            
            dataset=find_value(logdir_list, logtype)
            aug=find_value(logdir_list, dataset)
            model=find_value(logdir_list, aug)
            epoch=find_value(logdir_list, model)
            batchsize=find_value(logdir_list, epoch)
            worker=find_value(logdir_list, batchsize)
                             
            parse_filename=f"{logtype}_{dataset}_{model}_{aug}_{worker}_{epoch}_{batchsize}_cpuutil"
            
            logfile=logdir +"/"+ output_file
            breakdown_parsed_log = []
            single_log = []
            socket0_util_total=0
            socket1_util_total=0
            cpu_util_total=0
            for i in range(24):
                single_log.append(0)
            for line in open(logfile, 'r').readlines():
                if line.find(f'python') != -1 or line.find(f'/home/chanho/anaconda3/envs/torch1/bin') != -1: # start log
                    replace_txt=line.replace('\n','')
                    replace_txt=replace_txt.replace('[','')
                    replace_txt=replace_txt.replace(']','')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
                    info[-1] = info[-1].replace('\n','')
                    info[-1] = info[-1].replace("/home/chanho/anaconda3/bin/",'')
                    info[-1] = info[-1].replace("/home/chanho/anaconda3/envs/torch1/bin",'python')    
                    info[-1] = info[-1].replace("/usr/bin/python3",'python')
                    try:
                        cpu_util = float(find_value(info,"python",-7))
                        cpu_util_total += cpu_util
                        cpu_num = int(find_value(info,"python",-2))
                        single_log[cpu_num]=cpu_util
                    except:
                        print(info)
                        break
                    if cpu_num < 12:
                        socket0_util_total += cpu_util
                    else:
                        socket1_util_total += cpu_util
                elif line.find(f'Task') != -1: # start log
                    single_log.extend([socket0_util_total,socket1_util_total,cpu_util_total])
                    breakdown_parsed_log.append(single_log)
                    single_log = []
                    for i in range(24):
                        single_log.append(0)
                    socket0_util_total=0
                    socket1_util_total=0
                    cpu_util_total = 0
                    # append
                else:
                    pass

            breakdown_df = pd.DataFrame(breakdown_parsed_log,
                              columns=breakdown_col_name)
            breakdown_df.dropna().to_csv(parsed_dir+"/"+parse_filename+".csv", sep=',', na_rep='NA')

In [4]:
parser(dir_names, datasets)